In [15]:
import json
import pandas as pd
import numpy as np
from IPython.display import IFrame, display
    
def float_formatter(x): return '{:.3f}'.format(x)

def highlight_max(s):
    if (s.dtype == object):
        is_max = [False for _ in range(s.shape[0])]
    else:
        is_max = s == s.max()
    return ['background: lightgreen' if cell else '' for cell in is_max]

wanted_keys = ("internal", "external")
dictfilt = lambda x, y: dict([ (i,'{:.3f}'.format(float(x[i]))) for i in x if i in set(y) ])

def retrieve_single_result_cnv():
    f = open(f'result/cnv/cnv.json')
    data = json.load(f)
    new_dict = {}
    new_dict['cnv'] = {}
    new_dict['cnv']['spearman'] = dictfilt(data['spearman'], wanted_keys)
    return pd.DataFrame(new_dict)

def print_best_estimator(dataset, feature_selection):
    f = open(f'result/{dataset}/{dataset}.json')
    data = json.load(f)
    print(data[feature_selection]['best_estimator'])
    
def retrieve_results():
    new_dict = {}
    for dataset in ['miRNA', 'proteins', 'mRNA', 'concat']:
        new_dict[dataset] = {}
        f = open(f'result/{dataset}/{dataset}.json')
        data = json.load(f)
        for metodo in ['mannwhitneyu','mannwhitneyu_with_umap', 'boruta', 'mine', 'mrmr']:
            if metodo in data:
                new_dict[dataset][metodo] = dictfilt(data[metodo], wanted_keys)
            else: 
                new_dict[dataset][metodo] = -1
    return pd.DataFrame(new_dict)

def load_filtered_normalize_data():
    dataset_dict = ['cnv','mirna', 'mrna', 'proteins']
    dict_test = {}
    for data in dataset_dict:
        with open('filtered_normalize_data/'+data+'.json', 'r') as fp:
            dataset = pd.read_json(fp)
            dict_test[data] = dataset
    return dict_test

def load_data_after_r_function():
    dataset_dict = ['cnv','mirna', 'mrna', 'proteins', 'concat']
    dict_test = {}
    for data in dataset_dict:
        with open('elaborated_data_with_r_function/r_'+data+'_removed.json', 'r') as fp:
            dataset = pd.read_json(fp)
            dict_test[data] = dataset
    return dict_test

def retrieve_results_internally(lista_metodi, lista_dataset):
    new_dict = {}
    for dataset in lista_dataset:
        new_dict[dataset] = {}
        for metodo in lista_metodi:
            f = open(f'result_new_experiment_internally/{metodo}/{dataset}.json')
            data = json.load(f)
            #dictfilt(data[metodo], wanted_keys)
            new_dict[dataset][metodo] = dictfilt(data[metodo], wanted_keys)
    return pd.DataFrame(new_dict)

lista_metodi = ['r_function_intrinsic_with_tsne', 'r_function_intrinsic_with_umap',
                'r_function_with_mannwhitney', 'r_function_with_mannwhitney_boruta',
                'r_function_with_mannwhitney_mrmr']
lista_dataset = ['proteins', 'cnv', 'mrna', 'mirna', 'concat']

lista_metodi_ext = ['r_function_intrinsic_with_tsne', 'r_function_intrinsic_with_umap']

def retrieve_results_externally(lista_metodi_ext, lista_dataset):
    new_dict = {}
    for dataset in lista_dataset:
        new_dict[dataset] = {}
        for metodo in lista_metodi_ext:
            f = open(f'result_new_experiment_externally/{metodo}/{dataset}.json')
            data = json.load(f)
            #float(data[metodo]['internal'])
            new_dict[dataset][metodo] = dictfilt(data[metodo], wanted_keys)
    return pd.DataFrame(new_dict)

datasets = load_filtered_normalize_data()

data_after_r_function = load_data_after_r_function()

dataset_concat = pd.concat([datasets['cnv'], datasets['proteins'], datasets['mrna'], datasets['mirna']], axis=1)
datasets['concat'] = dataset_concat

# Osservazioni iniziali

1) Inizialmente tutti i i dati sono stati normalizzati usando MinMaxScaler e filtrati scartando le feature con bassa variabilità ( deviazione standard < 0.05)
3) Il modello usato è sempre RandomForest
3) Viene usata l'AUPRC come misuratore di performance. La baseline per il dataset fornito è 0.122807, ovvero il 12% dei pazienti sono positivi (77 su 627).
4) Viene sempre applicata una StratifiedKFold usando 10 fold.
5) I risultati sono sempre presentati con etichetta "external" e "internal". Questo sta a indicare quando è stata calcolata la AUPRC: nel primo caso le predizioni effettuate dal modello nei 10 fold vengono concatenate e una volta finite viene calcolata l'AUPRC totale; nel secondo caso invece ad ogni iterazione viene calcolata l'AUPRC e alla fine viene calcolata la media. Calcolando l'AUPRC esternamente sono anche in grado di avere un grafico, cosa che internamente non è possibile poiché avrei 10 grafici distinti.

### A cnv ho applicato spearman e scartato le feature altamente correlate tra di loro o bassamente correlate con la label

In [2]:
retrieve_single_result_cnv()

,cnv
spearman,"{'internal': '0.118', 'external': '0.100'}"


In [3]:
print_best_estimator('cnv', 'spearman')

RandomForestClassifier(criterion='entropy', max_features=None, max_leaf_nodes=5,
                       n_estimators=251)


### Figura disponibile solo per l'AUPRC "esterna"

In [4]:
filepath = "result/cnv/spearman_AUC_External.pdf"
IFrame(filepath, width=700, height=400)

# Risultati di proteins, miRNA e mRNA e concat (concatenazione di tutti i dataset)

In [5]:
retrieve_results()

,miRNA,proteins,mRNA,concat
mannwhitneyu,"{'internal': '0.210', 'external': '0.162'}","{'internal': '0.187', 'external': '0.137'}","{'internal': '0.216', 'external': '0.159'}","{'internal': '0.182', 'external': '0.152'}"
mannwhitneyu_with_umap,"{'internal': '0.414', 'external': '0.151'}","{'internal': '0.248', 'external': '0.108'}","{'internal': '0.386', 'external': '0.136'}","{'internal': '0.264', 'external': '0.111'}"
boruta,"{'internal': '0.140', 'external': '0.125'}","{'internal': '0.145', 'external': '0.119'}","{'internal': '0.157', 'external': '0.141'}","{'internal': '0.186', 'external': '0.140'}"
mine,"{'internal': '0.215', 'external': '0.174'}","{'internal': '0.186', 'external': '0.140'}","{'internal': '0.172', 'external': '0.134'}",-1
mrmr,"{'internal': '0.165', 'external': '0.130'}","{'internal': '0.177', 'external': '0.144'}",-1,-1


#### Per mRNA e concat in corrispodenza di mrmr c'è -1, questo perchè il calcolo richiedeva tantissimo tempo, sopratutto usando 10 fold

## Best estimator of the best combination of feature selection + extraction

In [6]:
print_best_estimator('proteins', 'mannwhitneyu_with_umap')

RandomForestClassifier(max_features='log2', max_leaf_nodes=5, n_estimators=51)


In [7]:
print_best_estimator('mRNA', 'mannwhitneyu_with_umap')

RandomForestClassifier(max_features='log2', n_estimators=51)


In [8]:
print_best_estimator('miRNA', 'mannwhitneyu_with_umap')

RandomForestClassifier(criterion='entropy', max_leaf_nodes=10, n_estimators=51)


In [9]:
print_best_estimator('concat', 'mannwhitneyu_with_umap')

RandomForestClassifier(criterion='entropy', max_features='log2',
                       n_estimators=101)


# Seconda tipologia di esperimenti

### Come prima cosa viene utlizzata una funzione che scarta le feature altamente correlate tra di loro usando "pearson" con un cutoff = 0.8. Vediamo quante feature vengono scartate:

In [10]:
for dataset in ['proteins', 'cnv', 'mirna', 'mrna', 'concat']:    
    print(f"{dataset}: feature prima dell applicazione della funzione: {len(datasets[dataset].columns)}")
    print(f"{dataset}: feature dopo l applicazione della funzione: {len(data_after_r_function[dataset].columns)}")
    print('\n')

proteins: feature prima dell applicazione della funzione: 216
proteins: feature dopo l applicazione della funzione: 216


cnv: feature prima dell applicazione della funzione: 24776
cnv: feature dopo l applicazione della funzione: 194


mirna: feature prima dell applicazione della funzione: 773
mirna: feature dopo l applicazione della funzione: 761


mrna: feature prima dell applicazione della funzione: 18465
mrna: feature dopo l applicazione della funzione: 18411


concat: feature prima dell applicazione della funzione: 44230
concat: feature dopo l applicazione della funzione: 20386




### La differenza tra internally e externally:
1) In entrambi i casi la funzione che calcola la correlazione tra le feature viene applicata sull'intero dataset, prima di darlo in pasto alla funzione che splitta i dati in fold e addestra
2) In 'internally' il calcolo della dimensione intrinseca viene effettuato sul train set del fold generato durante la StratifiedKFold e la dimensione calcolata viene usata per fare feature extraction sia sul train set che sul test set, usando tSNE e Umap.
3) In 'externally' invece la dimensione intrinseca viene calcolata esternamente alla funzione, quindi su tutto il dataset, e anche la feature extraction viene effettuata esternamente. Il dataset cosi ottenuto viene passato alla funzione che addestra.

### on fold

In [13]:
retrieve_results_internally(lista_metodi, lista_dataset)

,proteins,cnv,mrna,mirna,concat
r_function_intrinsic_with_tsne,"{'internal': '0.139', 'external': '0.126'}","{'internal': '0.151', 'external': '0.117'}","{'internal': '0.125', 'external': '0.121'}","{'internal': '0.134', 'external': '0.118'}","{'internal': '0.145', 'external': '0.116'}"
r_function_intrinsic_with_umap,"{'internal': '0.370', 'external': '0.187'}","{'internal': '0.182', 'external': '0.155'}","{'internal': '0.465', 'external': '0.208'}","{'internal': '0.277', 'external': '0.152'}","{'internal': '0.252', 'external': '0.145'}"
r_function_with_mannwhitney,"{'internal': '0.158', 'external': '0.150'}","{'internal': '0.142', 'external': '0.109'}","{'internal': '0.194', 'external': '0.182'}","{'internal': '0.198', 'external': '0.168'}","{'internal': '0.218', 'external': '0.151'}"
r_function_with_mannwhitney_boruta,"{'internal': '0.175', 'external': '0.143'}","{'internal': '-1.000', 'external': '-1.000'}","{'internal': '0.217', 'external': '0.163'}","{'internal': '0.158', 'external': '0.130'}","{'internal': '0.180', 'external': '0.144'}"
r_function_with_mannwhitney_mrmr,"{'internal': '0.177', 'external': '0.144'}","{'internal': '0.146', 'external': '0.110'}","{'internal': '0.221', 'external': '0.200'}","{'internal': '0.200', 'external': '0.161'}","{'internal': '0.204', 'external': '0.170'}"


### whole dataset (tranductive)

In [16]:
retrieve_results_externally(lista_metodi_ext, lista_dataset)

,proteins,cnv,mrna,mirna,concat
r_function_intrinsic_with_tsne,"{'internal': '0.138', 'external': '0.115'}","{'internal': '0.125', 'external': '0.124'}","{'internal': '0.110', 'external': '0.102'}","{'internal': '0.159', 'external': '0.125'}","{'internal': '0.135', 'external': '0.113'}"
r_function_intrinsic_with_umap,"{'internal': '0.708', 'external': '0.697'}","{'internal': '0.546', 'external': '0.559'}","{'internal': '0.639', 'external': '0.639'}","{'internal': '0.773', 'external': '0.775'}","{'internal': '0.677', 'external': '0.682'}"
